# Script to test the cooling function works properly.

We take a dummy halo of given mass, temperature and size, then determine the mean density.

Then
(i) cool assuming a uniform halo
(ii) cool using a dynamical time prescription

In [ ]:
import astropy.constants as c
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
import h5py
h5py.enable_ipython_completer()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')
import sys

In [ ]:
# Cosmological parameters
H0=70. # in km/s/Mpc
omega_m=0.3
cosmology = FlatLambdaCDM(H0=H0, Om0=omega_m, Tcmb0=2.725)
mumH=1e-27*u.kg
f_baryon=0.155
n2_ratio=0.25  # n_e n_i/n_{tot}^2
rhobar=200.*cosmology.critical_density(0)

# Halo parameters
# This is a massive halo at z=0: will not cool very much
mass=1e15*c.M_sun
print('Mass = {:.3g}'.format(mass.to(c.M_sun)))
radius=(3*mass/(4*np.pi*rhobar))**(1/3)
print('Radius = {:.3g}'.format(radius.to(u.Mpc)))
v_vir=np.sqrt(c.G*mass/radius)
print('Virial speed = {:.3g}'.format(v_vir.to(u.km/u.s)))
tau_dyn=radius/v_vir
print('Dynamical time = {:.3g}'.format(tau_dyn.to(u.yr)))
temperature=(mumH*c.G*mass)/(2*c.k_B*radius)
print('Temperature = {:.3g}'.format(temperature.to(u.K)))
Z=0.02 # Probably too high for a cluster, but it will do.
print('Metallicity = {:.3g}'.format(Z))

# Plotting parameters
t_end=1e12*u.yr # Cooling time very long @z=0 so have to extend to future
timestep=1e8*u.yr

In [ ]:
# Now some code to emulate what py-galaxies is doing.

# Imports of py-galaxies python routines
PYTHON_DIR='../code-python'
sys.path.insert(1,PYTHON_DIR)

# The parameter class, used to store run-time parameters
from parameters import C_parameters

# Read in parameters from yaml input files
# This file has the same parameter choices as above.
FILE_PARAMETERS='../input/input.yml'
# and this one we will ignore
FILE_OPTIONS_LIST='../input/available_options.yml'
parameters=C_parameters(FILE_PARAMETERS,FILE_OPTIONS_LIST)
parameters.cooling_function_file='../'+parameters.cooling_function_file

# Import astrophysics modules (could be run-time parameter dependent)
import cooling
cooling_table = cooling.C_cooling(parameters)
from cooling import F_cooling_SIS,F_get_metaldependent_cooling_rate

## Uniform density

The mass cooling rate per unit volume is given by
$${5k_BT\over2\mu m_\mathrm{H}}\dot\rho = -n_en_i\Lambda(T,Z).$$
Using 
$$n_e n_i=\left(\rho\over\mu m_\mathrm{H}\right)^2{n_en_i\over n_\mathrm{tot}^2},$$
where $n_\mathrm{tot}=\rho/\mu m_\mathrm{H}$, this becomes
$$\dot\rho = -{n_en_i\over n_\mathrm{tot}^2}{2\Lambda(T,Z)\over 5\mu m_\mathrm{H}k_B T}\,\rho^2.$$
Now we assume that the only variable is $\rho$ and set $f_g=\rho/\rho_0$, where the subscript zero refers to the initial value.  Then
$$\dot f_g= -{f_g^2\over\tau_\mathrm{cool}},$$
where
$$\tau_\mathrm{cool}={n_\mathrm{tot}^2\over n_en_i}{5\mu m_\mathrm{H}k_B T\over 2\rho_0\Lambda(T,Z)}$$ is a constant with dimensions of time.

The solution to this equation with $f_g=1$ at $\tau=0$ is
$$f_g={1\over 1+(t/\tau_\mathrm{cool})}.$$

In [ ]:
# The following routine uses py-galaxies format to implement the above
# Note that we normalise to the mean halo mass, not the mean gas mass
def F_cooling_test(mass,half_mass_radius,mass_gas,mass_metals_gas,temp_start,temp_end,dt,cooling_table):
    """
    Implements the isothermal cooling model assuming unifrm density, as a test
    """
    # Not sure if subhalo virial temperature can ever exceed that of the halo that it is in.
    # If it can, trap out before call to this subroutine, so raise error here
    if temp_end >= temp_start:
        print('cooling:F_cooling_SIS: mass, temp_start, temp_end =',mass,temp_start,temp_end)
        return mass_gas
    
    # Determine cooling function
    log10_Z = max(-10.,np.log10(mass_metals_gas/mass_gas))
    # Cooling rate per unit density of electrons & ions
    Lambda = F_get_metaldependent_cooling_rate(np.log10(temp_start),log10_Z,cooling_table)
    # Could save a little time in defining a conversion factor for half_mass_radius**3/mass in halos/subhalos.
    # (Because we execute the cooling every mini-step).
    tau_cool = half_mass_radius**3*(temp_start-temp_end)/(mass*Lambda)
    # tau_cool as defined in L-Galaxies is 3x larger than we need here
    tau_cool /=  3. 
    # The input file uses l-galaxies mode which gives a cooling time based on 3/2 not 5/2, so tau_cool needs to be increased
    tau_cool *= 5./3.

    # Cooling at constant temperature, but allowing density to vary: see documentation.
    # The gas fraction here is relative to the halo mass (= 200 times the critical density in Millennium/SIS)
    fg0 = mass_gas/mass  # This line should 
    fg = fg0-fg0**2*dt/tau_cool
    
    return fg*mass


In [ ]:
# Analytic solution, assuming that halo is uniform density
# This does NOT correspond to L-Galaxies!!

Lambda = cooling.F_get_metaldependent_cooling_rate(np.log10(temperature/parameters.units_temperature_internal),np.log10(Z),cooling_table)
# Undo the py-galaxies scaling
Lambda=Lambda*parameters.c_cooling
Lambda = Lambda * parameters.units_energy_internal * parameters.units_length_internal**3 / parameters.units_time_internal
# I have checked that this value matches that in the plot of the cooling function
print('Lambda = {:.3g}'.format(Lambda.to(u.erg*u.cm**3/u.s)))

# Cooling time.  This uses the mean gas density
tau_cool=5*mumH*c.k_B*temperature/(2.*n2_ratio*rhobar*f_baryon*Lambda)
print('tau_cool = {:.3g}'.format(tau_cool.to(u.yr)))

# Generate points for plot
t=np.arange(0,1.000001,timestep/t_end)*t_end
y_anal=1/(1+t/tau_cool)

In [ ]:
## py-galaxies solution
# First change parameters to py-galaxies units
t_step=(t_end/parameters.units_time_internal).si.value
# Quantities needed for py-galaxies cooling
mass_halo=(mass/parameters.units_mass_internal).si.value
mass_gas=(mass*f_baryon/parameters.units_mass_internal).si.value
mass_metals_gas=mass_gas*Z
half_mass_radius=0.5*(radius/parameters.units_length_internal).si.value
# Why do we need tau_dyn?  I think because get different answer if cooling time is short
temp_start=(temperature/parameters.units_temperature_internal).si.value
temp_end=0.

# Then continue with py-galaxies code
n_dt=int(t_step*1.000001/parameters.timestep_halo)+1
dt=t_step/n_dt
y_pygal=np.zeros(n_dt)
y_pygal[0]=mass_gas
for i_dt in range(1,n_dt):
    mass_gas=F_cooling_test(mass_halo,half_mass_radius,mass_gas,mass_metals_gas,temp_start,temp_end,dt,cooling_table)
    y_pygal[i_dt]=mass_gas
    mass_metals_gas=mass_gas*Z
# Convert mass_gas_hot to fractional change in gas mass
y_pygal=y_pygal/y_pygal[0]

# plot
plt.figure(figsize=[12,8])
plt.plot(t,y_anal,'-',label='analytic solution')
plt.plot(t,y_pygal,'-',label='py-gal solution')
plt.legend()

## Isothermal model

Cooling gas within the radius for which t_cool=tau_dyn on a dynamical time.

The theory is in the cooling paper.  It gives an expression for long cooling times ($\tau_\mathrm{cool}>f_{g0}\tau_\mathrm{dyn}$) of
$$f_g=f_{g0}\left(1+\left(\tau_\mathrm{dyn}f_{g0}\over\tau_\mathrm{cool}\right)^{1\over2}{\Delta t\over2\tau_\mathrm{dyn}}\right)^{-2},$$
where $\tau_\mathrm{cool}$ is as above and $\tau_\mathrm{dyn}=R/v_\mathrm{vir}$, where $R$ is the outer radius of the halo and $v_\mathrm{vir}$ is its virial speed.

Note that the gas fraction here, $f_g$, is measured with respect to the halo mass.


In [ ]:
# Analytic solution

Lambda = cooling.F_get_metaldependent_cooling_rate(np.log10(temperature/parameters.units_temperature_internal),np.log10(Z),cooling_table)
# Undo the py-galaxies scaling
Lambda=Lambda*parameters.c_cooling
Lambda = Lambda * parameters.units_energy_internal * parameters.units_length_internal**3 / parameters.units_time_internal
# I have checked that this value matches that in the plot of the cooling function
print('Lambda = {:.3g}'.format(Lambda.to(u.erg*u.cm**3/u.s)))

# Cooling time.
tau_cool=5*mumH*c.k_B*temperature/(2.*n2_ratio*rhobar*Lambda)
# Correct by 3./5. as input parameter file for py-gal uses L-Galaxies and thus 3/2kT rather than 5/2kT
tau_cool *= 3./5.
# Correct by 3. because tau_cool as defined in cooling paper is 3x larger than defined above
tau_cool *= 3.

print('tau_cool = {:.3g}'.format(tau_cool.to(u.yr)))

# Dynamical time
print('tau_dyn = {:.3g}'.format(tau_dyn.to(u.yr)))

fg0 = f_baryon
t_end=1e11*u.yr
t_step=1e9*u.yr

# Generate points for plot
t=np.arange(0,1.000001,timestep/t_end)*t_end
y_anal=1/(1+0.5*np.sqrt(tau_dyn*fg0/tau_cool)*t/tau_dyn)**2

In [ ]:
## py-galaxies solution
# First change parameters to py-galaxies units
t_step=(t_end/parameters.units_time_internal).si.value
# Quantities needed for py-galaxies cooling
mass_halo=(mass/parameters.units_mass_internal).si.value
mass_gas=(mass*f_baryon/parameters.units_mass_internal).si.value
mass_metals_gas=mass_gas*Z
half_mass_radius=0.5*(radius/parameters.units_length_internal).si.value
tau_dyn_internal=(tau_dyn/parameters.units_time_internal).si.value
temp_start=(temperature/parameters.units_temperature_internal).si.value
temp_end=0.

# Then continue with py-galaxies code
n_dt=int(t_step*1.000001/parameters.timestep_halo)+1
dt=t_step/n_dt
y_pygal=np.zeros(n_dt)
y_pygal[0]=mass_gas
for i_dt in range(1,n_dt):
    mass_gas=F_cooling_SIS(mass_halo,tau_dyn_internal,half_mass_radius,mass_gas,mass_metals_gas,temp_start,temp_end,dt,cooling_table)
    y_pygal[i_dt]=mass_gas
    mass_metals_gas=mass_gas*Z
# Convert mass_gas_hot to fractional change in gas mass
y_pygal=y_pygal/y_pygal[0]

# plot
plt.figure(figsize=[12,8])
plt.plot(t,y_anal,'-',label='analytic solution')
plt.plot(t,y_pygal,'--',label='py-gal solution')
plt.xlabel(r'time / yr')
plt.ylabel(r'$M_\mathrm{gas}/(f_\mathrm{baryon}\,M_\mathrm{halo})$')
plt.title(r'$\tau_\mathrm{cool}>f_\mathrm{gas}\tau_\mathrm{dyn}$')
plt.legend()
plt.savefig('cooling_test_iso_long.png',bbox_inches='tight')